In [1]:
# 01 - Import Libraries
import torch
from torch import nn
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import PIL
from torch.utils.data import DataLoader
from timeit import default_timer as timer
from tqdm.auto import tqdm
RANDOM_SEED = torch.manual_seed(42)

In [2]:
# Necessary Functions
device = "cuda" if torch.cuda.is_available() else "cpu"
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_true)) * 100
    return acc

def train_timer(start:float, end:float, device: torch.device=None):
    total_time = end-start
    print(f"Device:{device}:{total_time:.2f}sec")
    return total_time

def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn):
    model.eval()
    total_loss, total_acc = 0, 0

    with torch.inference_mode():
        for img, labels in tqdm(data_loader):
            img, labels = img.to(device), labels.to(device)  
            preds = model(img)
            loss = loss_fn(preds, labels)
            acc = accuracy_fn(labels, preds.argmax(dim=1))

            total_loss += loss.item()
            total_acc += acc

    avg_loss = total_loss / len(data_loader)
    avg_acc = total_acc / len(data_loader)

    return {
        "model_name": model.__class__.__name__,
        "model_loss": avg_loss,
        "model_acc": avg_acc
    }

def make_predictions(model: torch.nn.Module, data: list, device: torch.device = device):
    pred_probs = []
    model.eval()
    with torch.inference_mode():
        for sample in data:
            sample = torch.unsqueeze(sample, dim=0).to(device) 

            pred_logit = model(sample)

            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0) # note: perform softmax on the "logits" dimension, not "batch" dimension (in this case we have a batch size of 1, so can perform on dim=0)

            pred_probs.append(pred_prob.cpu())
            
    return torch.stack(pred_probs)

In [3]:
# 02 - Datasets
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform= None,
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform= None,
)

In [4]:
# 03 - DataLoad
class_names = train_data.classes
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=32,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                              batch_size=32,
                              shuffle=False)
train_features_batch, train_labels_batch = next(iter(train_dataloader))


In [ ]:
# 04 - CNN Model (TinyVGG)
class FashionMNISTModelV2(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3,
                      stride=1, 
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7, 
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        return self.classifier(self.block_2(self.block_1(x)))

torch.manual_seed(42)
model_2 = FashionMNISTModelV2(input_shape=1, 
    hidden_units=10, 
    output_shape=len(class_names)).to(device)
model_2

FashionMNISTModelV2(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=490, out_features=10, bias=True)
  )
)

In [7]:
# 05 - Loss Fnc. & Optimizer
loss_fn2 = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(), lr=0.1)

In [ ]:
# 06 - Training & Testing
RANDOM_SEED
train_time_start = timer()
epochs = 3

for epoch in range(epochs):
    print(f"Epoch: {epoch + 1}")
    model_2.train()
    epoch_loss = 0

    for batch, (img, labels) in enumerate(tqdm(train_dataloader)):
        img, labels = img.to(device), labels.to(device)

        train_preds = model_2(img)
        train_loss = loss_fn2(train_preds, labels)
        epoch_loss += train_loss.item()

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if batch % 400 == 0:
            seen = batch * len(img)
            total = len(train_dataloader.dataset)
            print(f"Looked at: {seen}/{total} samples")

    epoch_loss /= len(train_dataloader)

    # Testing
    model_2.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for img, labels in test_dataloader:
            img, labels = img.to(device), labels.to(device)

            test_preds = model_2(img)
            test_loss += loss_fn2(test_preds, labels).item()
            test_acc += accuracy_fn(labels, test_preds.argmax(dim=1))

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)

    print(f"Train Loss: {epoch_loss:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

train_time_end = timer()
total_train_time_model_1 = train_timer(train_time_start, train_time_end, device=device)

Epoch: 1


  0%|          | 0/1875 [00:00<?, ?it/s]

Looked at: 0/60000 samples
Looked at: 12800/60000 samples
Looked at: 25600/60000 samples
Looked at: 38400/60000 samples
Looked at: 51200/60000 samples
Train Loss: 0.6581 | Test Loss: 0.4297 | Test Acc: 84.1953
Epoch: 2


  0%|          | 0/1875 [00:00<?, ?it/s]

Looked at: 0/60000 samples
Looked at: 12800/60000 samples
Looked at: 25600/60000 samples
Looked at: 38400/60000 samples
Looked at: 51200/60000 samples
Train Loss: 0.3719 | Test Loss: 0.3835 | Test Acc: 86.2620
Epoch: 3


  0%|          | 0/1875 [00:00<?, ?it/s]

Looked at: 0/60000 samples
Looked at: 12800/60000 samples
Looked at: 25600/60000 samples
Looked at: 38400/60000 samples
Looked at: 51200/60000 samples
Train Loss: 0.3364 | Test Loss: 0.3596 | Test Acc: 85.9724
Device:cpu:7663.90sec


In [ ]:
# 07 - Result Evaluation
RANDOM_SEED
model_1_results = eval_model(model_2, test_dataloader, loss_fn2, accuracy_fn)
model_1_results


  0%|          | 0/313 [00:00<?, ?it/s]

{'model_name': 'FashionMNISTModelV2',
 'model_loss': 0.35959244219544595,
 'model_acc': 85.97244408945687}

In [ ]:
# 08 - Saving the model
torch.save(model_2.state_dict(), "Models/model_2.pth")
print("Model saved to model_2.pth")

Model saved to model_2.pth
